In [29]:
import pandas as pd
from music21 import converter, instrument, note, chord
from sklearn.preprocessing import LabelEncoder
import itertools

In [30]:
path = 'data\maestro-v3.0.0.csv'
df = pd.read_csv(path)

In [31]:
df = df.sort_values(by='duration').iloc[:10]

In [32]:
def extraction_notes(file):
    midi = converter.parse(file)
    L_notes = []
    for e in midi.recurse().notes:
        if e.isNote:
            x = [e.pitch.midi]+9*[-1]
            L_notes.append(x)
        elif e.isChord:
            x = [p.midi for p in e.pitches]
            while len(x)<10:
                x.append(-1)
            L_notes.append(x)
    return L_notes

def ex_notes_tab(df,col_out):
    df['midi_file_data'] = 'data/'+df['midi_filename']
    df[col_out] = df['midi_file_data'].apply(extraction_notes)
    return df

def colonnes(df):
    df_expanded = df['X'].apply(pd.Series)
    df_expanded['y'] = df['y']

    return df_expanded

def sequence_list(L):
    sequence_length = 500

    input_sequences = []
    output_notes = []

    for i in range(len(L) - sequence_length):
        seq_in = L[i:i + sequence_length]  
        seq_out = L[i + sequence_length]  
        input_sequences.append(seq_in)
        output_notes.append(seq_out)

    return input_sequences,output_notes

def sequence(dff):
    X,y = [],[]
    for k in dff['notes'].to_list():
        xtemp,ytemp = sequence_list(k)
        X.append(xtemp)
        y.append(ytemp)
    X=list(itertools.chain(*X))
    y=list(itertools.chain(*y))

    return pd.DataFrame({"X":X,"y":y})
    

In [37]:
def preprocessing(df,split):
    dff = df[df['split']==split]
    print(dff.head())
    dff = ex_notes_tab(dff,'notes')
    print(dff.head())
    return colonnes(sequence(dff))

In [39]:
dff = df[df['split']=='test']

In [42]:
dff = ex_notes_tab(dff,'notes')
dff

C:\Users\ccdeb\AppData\Local\Temp\ipykernel_12600\1756167662.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['midi_file_data'] = 'data/'+df['midi_filename']
C:\Users\ccdeb\AppData\Local\Temp\ipykernel_12600\1756167662.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[col_out] = df['midi_file_data'].apply(extraction_notes)


,canonical_composer,canonical_title,split,year,midi_filename,audio_filename,duration,midi_file_data,notes
92,Domenico Scarlatti,Sonata K. 525,test,2008,2008/MIDI-Unprocessed_09_R3_2008_01-07_ORIG_MI...,2008/MIDI-Unprocessed_09_R3_2008_01-07_ORIG_MI...,65.948118,data/2008/MIDI-Unprocessed_09_R3_2008_01-07_OR...,"[[60, -1, -1, -1, -1, -1, -1, -1, -1, -1], [65..."


In [43]:
sequence(dff)

,X,y
